**Exercise 1**

What is the wild-type production of your group’s compound?

In [5]:
from cobra.io import read_sbml_model

#Importar o modelo utilizando Cobrapy
model = read_sbml_model('iML1515.xml')

#Condições ambientais definidas
envcond = {'EX_glc__D_e': (-15.0, 100000.0),
           'EX_o2_e':(0,1000)}

#EX_mal__L_e -> L-malate
#EX_mal__D_e -> D-malate

from mewpy.simulation import get_simulator

#Realização do FBA
simul = get_simulator(model, envcond=envcond)
result = simul.simulate(method='FBA')

#Obtenção do valor dos fluxos WT
print('L-Malate wild type flux: ', result.fluxes['EX_mal__L_e'])
print('D-Malate wild type flux: ',result.fluxes['EX_mal__D_e'])

L-Malate wild type flux:  0.0
D-Malate wild type flux:  0.0


**Exercício 2**

What are the maximum compound production capabilities?

In [4]:
#Importar o modelo
from cobra.io import read_sbml_model
model = read_sbml_model('iML1515.xml')

print('------------------------------------------')
print('Implementação no Mewpy: ')
print('------------------------------------------')

#Definir condições ambientais
envcond = {'EX_glc__D_e': (-15.0, 100000.0),
           'EX_o2_e':(0,1000)}

#Carregar modelo no Mewpy
from mewpy.simulation import get_simulator
simul = get_simulator(model, envcond=envcond)

#Realizar FVA no Mewpy
print(simul.FVA(reactions=['EX_mal__L_e', 'EX_mal__D_e'], format=('df')))


------------------------------------------
Implementação no Mewpy: 
------------------------------------------
   Reaction ID  Minimum   Maximum
0  EX_mal__L_e      0.0  1.750857
1  EX_mal__D_e      0.0  0.000000
------------------------------------------
Implementação no Cobrapy: 
------------------------------------------
             minimum       maximum
EX_mal__L_e      0.0 -1.293965e-13
             minimum  maximum
EX_mal__D_e      0.0      0.0


**Exercício 3**

Use different optimization objective functions to improve the production of the compound, considering that cells have evolved for maximum growth.

*a)* Evaluate if single gene deletions enhance the production of the compound. Rank the mutants obtained according to the compound production capacity and growth performance.

Using 50 generations and a max candidate size of 1 in the package Mewpy, a gene knockout problem was formulated using the objective function WYIELD, which was the only objective function implemented in the Mewpy package to yield results. As such, it was implemented in the two required evaluator, in one with a alfa value of 1 and the other with the default alfa value of 0.3.
40 unique single gene deletions were obtained, with corresponding best Malate production flux values, all giving similar results. These results however were below maximum Mewpy FVA of the original model, and we could not conclude if these deletions enhance te production of the given compound.
The mutants are ranked according to their production capacity in the Single_gene_deletions tab of the Lista_final_otimizações.xlsx file. Biomass flux values of these mutantes was computed (Flux_deletions.py), and was the same as the WT FBA across all mutants.
The code used to obtain these results is present in the BS_1_3a.py file.

*b)* Determine the best strategy, up to five modifications, to improve the compound production.

Using Mewpy, four optimization problems were formulated and implemented: Gene Knockout (GKO), Gene Under/Overexpression (GOU), Reaction Knockout (RKO) and Reation Under/Overexpression (ROU).
Across all problems the WYIELD objective function was the only one to yield results. The maximum number of modifications was set at 5 modifications. 50 generations were produced except to the GOU problem were only 20 were produced due to computational strain.
After analysing the optimization results using Excel, the strategy which produced the greatest compound production capacity was a ROU problem solution (referenciar screenshot).
However, screening the obtained dataset using the criterion of the smallest number of modifications coupled with the greatest compound production capacity, a single gene under expression of the b3360 gene was flagged as the best strategy to improve the compound production.
The code used to implement this problems is in the BS_3b_GKO.py, BS_3b_GOU.py, BS_3b_RKO.py e BS_3b_ROU.py. Os datasets obtidos estão respetivamente nas tabs GKO_list, GOU_list, RKO_list e ROU_list fo ficheiro Lista_final_otimizações.xlsx.


**Exercise 4**
Analyse the strategies proposed by your optimization, from the metabolic viewpoint. Would it be feasible to implement this strategy in the lab? Why?

From a metabolic point of view the best strategy is the one who leads to the biggest compound production value. However, from a laboratorial viewpoint, the most feasible optimization is the one who leads to the greatest compound production values while minimizing the number of modifications.
Also, it is easier to implement gene and reaction knockouts in the lab, than to implement gene and reaction under/over expression.
As such, analysing our results, we conclude that the b3360 gene under expression is the most feasible optimization strategy for malate production in our model, because it is the mutant with the greatest compound production value (above maximum FVA value) while only requiring a single modification, though it is harder to implement than a simple gene knockout mutant.


